In [1]:
!pip install streamlit pyngrok openpyxl --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 365.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [2]:
%%writefile clv_app.py
import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

st.set_page_config(page_title="Customer Value Predictor", layout="centered")
st.title("📈 Customer Value Prediction App")

# Step 1: Upload Excel
uploaded_file = st.file_uploader("📤 Upload your Excel file", type=["xlsx"])

if uploaded_file:
    try:
        # Step 2: Read and show column names
        df = pd.read_excel(uploaded_file)
        st.subheader("📋 Column Names Detected:")
        st.write(df.columns.tolist())

        # Step 3: Store cleaned data in a variable
        df_cleaned = df.dropna(subset=['Customer ID'])
        df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'], errors='coerce')
        df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]
        df_cleaned = df_cleaned[df_cleaned['Price'] > 0]
        df_cleaned['TotalPrice'] = df_cleaned['Quantity'] * df_cleaned['Price']

        # Store snapshot date and calculate RFM
        snapshot_date = df_cleaned['InvoiceDate'].max() + pd.Timedelta(days=1)

        rfm = df_cleaned.groupby('Customer ID').agg({
            'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
            'Invoice': 'nunique',
            'TotalPrice': 'sum'
        }).rename(columns={
            'InvoiceDate': 'Recency',
            'Invoice': 'Frequency',
            'TotalPrice': 'Monetary'
        })

        st.subheader("✅ Processed RFM Data:")
        st.write(rfm.head())

        # Step 4: Predict customer value
        X = rfm[['Recency', 'Frequency']]
        y = rfm['Monetary']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        st.subheader("📊 Model Evaluation")
        st.success(f"RMSE: {rmse:.2f}")
        st.success(f"R² Score: {r2:.2f}")

        if st.checkbox("Show Prediction Table"):
            st.write(pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}))

    except Exception as e:
        st.error(f"❌ Error: {e}")


Writing clv_app.py


In [4]:
!ngrok config add-authtoken 2zUv87gQUDDua9D97j5T8Nt0sd9_3ptdpiB8Xr2dHJDBaEtAv


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from pyngrok import ngrok
import time

# Kill previous processes and tunnels
!pkill streamlit
ngrok.kill()

# Start Streamlit app in background
!nohup streamlit run clv_app.py --server.enableCORS false --server.port 8501 &

# Wait for app to start
time.sleep(5)

# ✅ Correct way to connect to Streamlit using HTTP tunnel
public_url = ngrok.connect(8501, "http")
print(f"🌐 Your Streamlit app is live here: {public_url}")


nohup: appending output to 'nohup.out'
🌐 Your Streamlit app is live here: NgrokTunnel: "https://aaa31e8d6f51.ngrok-free.app" -> "http://localhost:8501"
